# 
## 실전 속의 디자인



### 팩토리 패턴
+ 파이썬은 클래스, 함수, 사용자 정의 객체 등의 역할이 구분되지 않으므로 필요하지 않음

### 생성 패턴
+ 객체 초기화를 위한 파라미터를 결정하거나 초기화에 필요한 관련 객체를 준비하는 등의 모든 관련 작업을 단순화하려는 것

#### 싱글턴과 공유 상태
+ 일반적인 상태에서 싱글턴은 사용하지 않은 것이 좋음
+ 객체 지향 소프트웨어를 위한 전역 변수의 한 형태이며 나쁜 습관일 확률이 높음
+ 파이썬에서 이를 해결하는 가장 쉬운 방법은 모듈을 사용하는 것. 여러 번 임포트하더라도 sys.modules에 로딩되는 것은 항상 하나임
+ 클래스 변수의 게터나 세트를 활용하거나 디스크립터 사용할 수도 있음

In [21]:
import logging

logging.basicConfig()
logger = logging.getLogger(__name__)


""" 게터/세터 구현 """
class GitFetcher:
    _current_tag = None

    def __init__(self, tag):
        self.current_tag = tag # 프로퍼티를 통해 클래스 변수와 연결되어 있음

    @property
    def current_tag(self):
        if self._current_tag is None:
            raise AttributeError("Tag is not initialized yet.")
        return self._current_tag 
    
    @current_tag.setter
    def current_tag(self, new_tag):
        self.__class__._current_tag = new_tag 
        
    def pull(self):
        logger.info(f"Pull at {self.current_tag}")

        
if __name__ == "__main__":
    f1 = GitFetcher(0.1)
    f2 = GitFetcher(0.2)
    f1.current_tag = 0.3
    
    f2.pull()
    f1.pull()

INFO:__main__:Pull at 0.3
INFO:__main__:Pull at 0.3


In [24]:
import logging

logging.basicConfig()
logger = logging.getLogger(__name__)


""" 디스크립터 구현 """
class SharedArrtibute:
    def __init__(self, initial_value=None):
        self.value = initial_value
        self._name = None
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        if self.value is None:
            raise AttributeError(f"{self._name} was never set")
        return self.value
    
    def __set__(self, instance, new_value):
        logger.info("calling __set__")
        self.value = new_value
        
    def __set_name__(self, owner, name):
        self._name = name
        
        
class GitFetcher:
    current_tag = SharedArrtibute()
    current_branch = SharedArrtibute()

    def __init__(self, tag, branch=None):
        self.current_tag = tag 
        self.__class__.current_branch = branch # __class__를 사용하면 디스크립터가 동작하지 않음

    def pull(self):
        logger.info(f"Pull at {self.current_tag}")
        return self.current_tag
    
if __name__ == "__main__":
    f1 = GitFetcher(0.1)
    f2 = GitFetcher(0.2)
    f1.current_tag = 0.3
    
    f2.pull()
    f1.pull()

INFO:__main__:calling __set__
INFO:__main__:calling __set__
INFO:__main__:calling __set__
INFO:__main__:Pull at 0.3
INFO:__main__:Pull at 0.3


### borg 패턴

+ 어쩔 수 없이 꼭 싱글턴을 사용해야 하는 상태에서의 대안
+ 같은 클래스의 모든 인스턴스가 모든 속성을 복제하는 객체를 만드는 것

In [26]:
class BaseFetcher:
    def __init__(self, source):
        self.source = source

        
class TagFetcher(BaseFetcher):
    _attributes = {} # borg 패턴의 특징,
    
    def __init__(self, source):
        self.__dict__ = self.__class__._attributes
        super().__init__(source)
        
    def pull(self):
        logger.info(f"Pull at tag {self.source}")
        return f"Tag = {self.source}"
    

class BranchFetcher(BaseFetcher):
    _attributes = {} # 사전은 레퍼런스 형태로 전달되는 변경 가능한 mutable 객체이므로
                     # 사전을 업데이트하면 모든 객체에 동일하게 업데이트 됨
    
    def __init__(self, source):
        self.__dict__ = self.__class__._attributes
        super().__init__(source)
        
    def pull(self):
        logger.info(f"Pull at branch {self.source}")
        return f"Branch = {self.source}"    

+ 실수로 사전과 관련된 로직을 추가하는 것을 방지하기 위해 믹스인 클래스를 사용해서 사전을 만드는 방법

In [69]:
## 공통 부분 시작 ##
class SharedAllMixin:
    def __init__(self, *args, **kwargs):
        try:
            self.__class__._attributes
        except AttributeError:
            self.__class__._attributes = {}

        self.__dict__ = self.__class__._attributes
        super().__init__(*args, **kwargs)


class BaseFetcher:
    def __init__(self, source):
        self.source = source
## 공통 부분 끝 ##

class TagFetcher(SharedAllMixin, BaseFetcher):
    def pull(self):
        logger.info("pulling from tag %s", self.source)
        return f"Tag = {self.source}"


class BranchFetcher(SharedAllMixin, BaseFetcher):
    def pull(self):
        logger.info("pulling from branch %s", self.source)
        return f"Branch = {self.source}"

    
if __name__ == "__main__":
    f1 = TagFetcher(0.1)
    f2 = TagFetcher(0.2)
    
    print(f2.__dict__)
    print(TagFetcher.mro())
    print(SharedAllMixin.__subclasses__())
    
    f2.pull()
    f1.pull()

{'source': 0.2}
[<class '__main__.TagFetcher'>, <class '__main__.SharedAllMixin'>, <class '__main__.BaseFetcher'>, <class 'object'>]


AttributeError: 'TagFetcher' object has no attribute '__subclasses__'

### 빌더

+ 필요로 하는 모든 객체를 직접 생성해주는 하나의 복잡한 객체를 만들어야 함
+ 한 번에 모든 것을 처리해주는 추상화를 해야 함

https://hoony-gunputer.tistory.com/entry/%EB%94%94%EC%9E%90%EC%9D%B8-%ED%8C%A8%ED%84%B4-Builder-Pattern-by-python-java
https://medium.com/modern-nlp/10-great-ml-practices-for-python-developers-b089eefc18fc

## 구조패턴

+ 여러 개의 객체를 조합하여 반복을 제거

### 어댑터 패턴
+ 두 가지 방법으로 구현 가능 
+ 첫 번째는 사용하려는 클래스를 상속 받는 클래스를 만드는 것
    + 상속은 얼마나 많은 외부 라이브러리를 가져올지 정확히 할기 어려우므로 is a 관계에 한정해서 적용하는 것이 바람직 함

In [41]:
class UsernameLookup:
    def search(self, user_namespace):
        logger.info("looking for %s", user_namespace)

In [42]:
class UserSource(UsernameLookup):
    def fetch(self, user_id, username):
        user_namespace = self._adapt_arguments(user_id, username)
        return self.search(user_namespace)

    @staticmethod
    def _adapt_arguments(user_id, username):
        return f"{user_id}:{username}"

+ 두 번째 방법은 컴포지션을 사용 
    + 더 나은 방법으로 소개함

In [45]:
class UserSource:
    def __init__(self, username_lookup: UsernameLookup) -> None:
        self.username_lookup = username_lookup

    def fetch(self, user_id, username):
        user_namespace = self._adapt_arguments(user_id, username)
        return self.username_lookup.search(user_namespace)

    @staticmethod
    def _adapt_arguments(user_id, username):
        return f"{user_id}:{username}"

### 컴포지트

+ 객체는 구조화된 트리 형태로 볼 수 있으며, 기본 객체는 리프 노드이고 컨테이너 객체는 중간 노드라 볼 수 있음
+ 클라이언트는 이 중 아무거나 호출하여 결과를 얻으려고 할 것임
+ 또한 컴포지트 객체도 클라이언트 처럼 동작함
+ 즉 리프 노드인지 중간 노드인지 상관없이 해당 요청을 관련노드가 처리할 수 있을 때 까지 계속 전달

In [54]:
from typing import Iterable, Union


class Product:
    def __init__(self, name, price):
        self._name = name
        self._price = price

    @property
    def price(self):
        return self._price


class ProductBundle:
    def __init__(
        self,
        name,
        perc_discount,
        *products: Iterable[Union[Product, "ProductBundle"]]
    ) -> None:
        self._name = name
        self._perc_discount = perc_discount
        self._products = products

    @property
    def price(self):
        total = sum(p.price for p in self._products)
        return total * (1 - self._perc_discount)
    
if __name__ == "__main__":

        electronics = ProductBundle(
            "electronics",
            0,
            ProductBundle(
                "smartphones",
                0.15,
                Product("smartphone1", 200),
                Product("smartphone2", 700),
            ),
            ProductBundle(
                "laptops",
                0.05,
                Product("laptop1", 700),
                Product("laptop2", 950),
            ),
        )
        tablets = ProductBundle(
            "tablets", 0.05, Product("tablet1", 200), Product("tablet2", 300)
        )
        total = ProductBundle("total", 0, electronics, tablets)
        logger.info(total.price)

INFO:__main__:2807.5


### 데코레이터 패턴

+ 파이선 데코레이터와는 다른 개념
+ 상속을 하지 않고도 객체의 기능을 동적으로 확장 할 수 있음
+ 동일한 인터페이스를 가지고 여러 단계를 거쳐 결과를 장식하거나 결합도 할 수 있는 또 다른 객체를 만드는 것
+ 이 객체들은 연결되어 있으며 각각의 객체는 본래 의도에 더해 새로운 기능이 추가될 수 있음

In [55]:
class DictQuery:
    def __init__(self, **kwargs):
        self._raw_query = kwargs

    def render(self) -> dict:
        return self._raw_query

In [56]:
class QueryEnhancer:
    def __init__(self, query: DictQuery):
        self.decorated = query

    def render(self):
        return self.decorated.render()


class RemoveEmpty(QueryEnhancer):
    def render(self):
        original = super().render()                     # 오리지널을 받아서
        return {k: v for k, v in original.items() if v} # 새로운 형식으로 리턴함


class CaseInsensitive(QueryEnhancer):
    def render(self):
        original = super().render()
        return {k: v.lower() for k, v in original.items()}

In [57]:
original = DictQuery(foo="bar", empty="", none=None, upper="UPPERCASE", title="Title")
new_query = CaseInsensitive(RemoveEmpty(original))

In [58]:
new_query.render()

{'foo': 'bar', 'upper': 'uppercase', 'title': 'title'}

+ 파이썬의 동적인 특성을 활용해 다른 방법으로 구현한 사례

In [65]:
from typing import Callable, Dict, Iterable

class QueryEnhancer:
    def __init__(
        self,
        query: DictQuery,
        *decorators: Iterable[Callable[[Dict[str, str]], Dict[str, str]]] # Callable[[input], return type]
    ) -> None:
        self._decorated = query
        self._decorators = decorators

    def render(self):
        current_result = self._decorated.render()
        for deco in self._decorators:
            current_result = deco(current_result)
        return current_result


def remove_empty(original: dict) -> dict:
    return {k: v for k, v in original.items() if v}


def case_insensitive(original: dict) -> dict:
    return {k: v.lower() for k, v in original.items()}

In [62]:
QueryEnhancer(original, remove_empty, case_insensitive).render()

{'foo': 'bar', 'upper': 'uppercase', 'title': 'title'}

### 파사드 패턴
+ 여러 객체가 다대다 관계를 이루며 상화작용을 하는 경우 사용됨
+ 각각의 객체에 대한 모든 연결을 만드는 대신 파사드 역할을 하는 중간 객체를 만들어 해결
+ 디렉토리의 패키지 빌드할 때 \_\_init\_\_.py 파일을 나머지 파일들과 함께 두는 것은 모듈의 루트오서 파사드와 같은 역할을 함
    + \_\_init\_\_.py 파일의 API가 유지되는 한 클라이언트에 영향을 주지 않게 됨

## 행동 패턴
+ 어떤 패턴을 사용하든지 간에 결국에는 중복을 피하거나 행동을 캡슐화하는 추상화를 통해 모델 간 결합력을 낮추는 방법

### 책임 연쇄 패턴

+ 후계자라는 개념이 추가됨, 현재 초리할 수 없을 때 대비한 다음 이벤트 객체를 의미
+ 직접 처리가 가능한 경우 결과를 반환하지만 처리가 불가능하면 후계자에게 전달하고 이 과정을 반복함

In [76]:
import re

class Event:
    pattern = None

    def __init__(self, next_event=None):
        self.successor = next_event

    def process(self, logline: str):
        if self.can_process(logline):
            return self._process(logline)

        if self.successor is not None:
            return self.successor.process(logline)

    def _process(self, logline: str) -> dict:
        parsed_data = self._parse_data(logline)
        return {
            "type": self.__class__.__name__,
            "id": parsed_data["id"],
            "value": parsed_data["value"],
        }

    @classmethod
    def can_process(cls, logline: str) -> bool:
        return cls.pattern.match(logline) is not None

    @classmethod
    def _parse_data(cls, logline: str) -> dict:
        return cls.pattern.match(logline).groupdict()


class LoginEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+login\s+(?P<value>\S+)")


class LogoutEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+logout\s+(?P<value>\S+)")


class SessionEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+log(in|out)\s+(?P<value>\S+)")

In [77]:
chain = LogoutEvent(LoginEvent())

In [78]:
chain.process("567: login User") # 로그 아웃이 먼저 실행됨

{'type': 'LoginEvent', 'id': '567', 'value': 'User'}

### 커맨트 패턴
<순서>
+ 실행될 명령의 파라미터들을 저장하는 객체를 만드는 것
+ 명령에 필요한 파라미터에 필터를 더하거나 제거하는 것처럼 상호작용할 수 잇는 메서드 제공
+ 마지막으로 실제로 작업을 수행할 객체를 만듬

### 상태 패턴
+ 상태별로 작은 객체를 만들어 각각의 객체가 적은 책임을 갖게 하도록 구현

In [96]:
import abc

class InvalidTransitionError(Exception):
    """Raised when trying to move to a target state from an unreachable source
    state.
    """


class MergeRequestState(abc.ABC):
    def __init__(self, merge_request):
        self._merge_request = merge_request

    @abc.abstractmethod
    def open(self):
        ...

    @abc.abstractmethod
    def close(self):
        ...

    @abc.abstractmethod
    def merge(self):
        ...

    def __str__(self):
        return self.__class__.__name__


class Open(MergeRequestState):
    def open(self):
        self._merge_request.approvals = 0

    def close(self):
        self._merge_request.approvals = 0
        self._merge_request.state = Closed # 상태를 전환하는 코드

    def merge(self):
        logger.info("merging %s", self._merge_request)
        logger.info("deleting branch %s", self._merge_request.source_branch)
        self._merge_request.state = Merged # 상태를 전환하는 코드


class Closed(MergeRequestState):
    def open(self):
        logger.info("reopening closed merge request %s", self._merge_request)
        self._merge_request.state = Open # 상태를 전환하는 코드

    def close(self):
        """Current state."""

    def merge(self):
        raise InvalidTransitionError("can't merge a closed request")


class Merged(MergeRequestState):
    def open(self):
        raise InvalidTransitionError("already merged request")

    def close(self):
        raise InvalidTransitionError("already merged request")

    def merge(self):
        """Current state."""


class MergeRequest:
    def __init__(self, source_branch: str, target_branch: str) -> None:
        self.source_branch = source_branch
        self.target_branch = target_branch
        self._state = None
        self.approvals = 0
        self.state = Open # 상태를 전환하는 코드

    @property
    def state(self):
        return self._state

    @state.setter
    def state(self, new_state_cls):
        print("calling setter", new_state_cls)
        self._state = new_state_cls(self)

    def open(self):
        return self.state.open()

    def close(self):
        return self.state.close()

    def merge(self):
        return self.state.merge()

    def __str__(self):
        return f"{self.target_branch}:{self.source_branch}"


In [97]:
mr = MergeRequest("develop", "master")
mr.open()
mr.approvals

calling setter <class '__main__.Open'>


0

In [98]:
mr.approvals = 3
mr.close()
mr.approvals

calling setter <class '__main__.Closed'>


0

In [99]:
mr.open()

INFO:__main__:reopening closed merge request master:develop


calling setter <class '__main__.Open'>


In [100]:
mr.merge()

INFO:__main__:merging master:develop
INFO:__main__:deleting branch develop


calling setter <class '__main__.Merged'>


In [101]:
mr.close()

InvalidTransitionError: already merged request

### Null 객체 패턴

+ 함수나 메서드는 일관된 타입을 잔환해야 한다는 것
+ 검색하는 사용자가 없다고 가정했을 때
    + 예외를 발생하거나
    + UserUnknow 타입을 반환
    + 어떤 경우에도 None을 반환하면 안됨
    + None이라는 문구는 방금 일어난 일에 대한 아무것도 설명해주지 않으며 호출자는 특별한 공지가 없으면 아무 생각없이 반환 객체에 대해 메서드를 호출할 것이므로 결국 AttributeError가 발생하게 된다.